In [11]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import time


client = MongoClient('localhost',27017)
db = client['vacancies_db']
hh_vac = db.hh_vac
# sj_vac = db.sj_vac

In [12]:
def hh_search(text, page, only_new = 'yes', hh_vac = hh_vac):

    user_agent = {'User-agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:108.0) Gecko/20100101 Firefox/108.0"}
    link_hh = ('https://kazan.hh.ru/search/vacancy?area=2&st=searchVacancy@enable_snippets=true&')
    #main_link = ('localhost/hh.html/')
    i = 0
    vacs = []

    for p in range(page):
        html = requests.get(f'{link_hh}&text={text}&page={p}',  headers = user_agent).text
        parsed_html = bs(html,'lxml')
        vac_list = parsed_html.findAll(class_="vacancy-serp-item-body__main-info")
        df = pd.DataFrame(columns=['name', 'salary_min', 'salary_max', 'url', 'site'])

        for vac in vac_list:
            #print(i)
            vac_data = {}

            main_info = vac.findChild()
            link = vac.find('a')
            if not link:
                url = 'nan'
                vac_data['name'] = ''
            else:
                vac_data['name'] = link.getText()
                url = link["href"].split('?')[0]
                vac_num = url.split('/')[3]



            # compensation = vac.findParent().find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            compensation = vac.find(
                attrs={"data-qa": "vacancy-serp__vacancy-compensation"}
            )
            
            salary = 0
            salary_min = float('nan')
            salary_max = float('nan')
            if not compensation:
                pass
            else:
                salary = compensation.getText()
                salary = re.sub(r"\s+", "", salary)
                if salary.find('от') >= 0 :
                    salary_min = int(re.findall(r'\d+',  salary)[0])

                elif salary.find('до') >= 0 :
                    salary_max = int(re.findall(r'\d+', salary)[0])

                elif salary.find('-') >= 0 :
                    salary = re.findall(r'\d+', salary)
                    salary_min = salary[0]
                    salary_max = salary[1]

                #print('sal', salary)
                #print('salary_min', salary_min)
                #print('salary_max', salary_max)
                vac_data['salary_min'] = salary_min
                vac_data['salary_max'] = salary_max
                vac_data['url'] = url
                vac_data['vac_num'] = vac_num
                vac_data['site'] = 'HeadHunter'

                vacs.append(vac_data)
                df.append(vac_data, ignore_index = True)
                if only_new == 'yes':
                  if find_in_db('hh', vac_num) == 0:
                    hh_vac.insert_one(vac_data)
                else:
                  # Не слишком продуктивно писать в базу по одной, зато доп. задержка для сайта
                  hh_vac.insert_one(vac_data)


            i += 1
            time.sleep(1)
    return(vacs)

In [13]:
def find_in_db(site, vac_num):
  exists = 0
  if site == 'hh':
    exists = hh_vac.count_documents({'vac_num': vac_num})
  return exists

def find_salary():
  salary = input('Введите минимальную зарплату: ')
  expensive_vacs = hh_vac.find({'salary_min':{'$gte':salary}}).sort('salary_min')
  print('Вакансии с зарплатой выше чем ', salary)
  for item in expensive_vacs:
    pprint(item)

find_salary()

Введите минимальную зарплату: 100000
Вакансии с зарплатой выше чем  100000


In [14]:
def search_vac(only_new ='yes'):

  text = input('Введите вакансию: ')
  text = text.replace(' ', '+')
  page = int(input('Введите кол-во страниц: '))
  #print(hh_search(text, page))
  hh_search(text, page, only_new)
  # sj_search(text, page, only_new)

  #print(sj_search(text, page))
  df1 = pd.DataFrame.from_dict(hh_search(text, page))
  # df2 = pd.DataFrame.from_dict(sj_search(text, page))
  df = pd.concat([df1], ignore_index=True)
  pprint(df1)
  #print(df2)
  return df

#df = pd.DataFrame(data=vacs)
#print(df)
print(search_vac('yes'))
#vacs_in_db = hh_vac.find()
#for item in vacs_in_db:
    #pprint(item)

Введите вакансию: Аналитик
Введите кол-во страниц: 20


C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

...

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local\Temp\ipykernel_2840\2163832178.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(vac_data, ignore_index = True)
C:\Users\dnr_z\AppData\Local

                                                  name  salary_min  \
0    Ведущий аналитик по мониторингу и анализу ассо...         NaN   
1    Менеджер по работе с маркетплейсами (OZON, Янд...     60000.0   
2                    Специалист по контекстной рекламе         NaN   
3                                  Помощник экономиста     60000.0   
4    Менеджер / руководитель направления товары для...         NaN   
..                                                 ...         ...   
180                                        Аналитик 1С         NaN   
181                            System Business Analyst         NaN   
182                            Бренд-менеджер ДЦ CHERY         NaN   
183                             Media buyer (MyTarget)    100000.0   
184               Функциональный архитектор (удаленно)         NaN   

     salary_max                                   url  vac_num        site  
0           NaN             https://adsrv.hh.ru/click    click  HeadHunter  
1    